In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 2025-08-25
# 프로젝트 01(기초 분석) 단계:
# - 급가속/급제동 이벤트를 "임계값 기준"으로 먼저 카운트하고
# - 주행 시간/거리 기반 빈도(events/min, events/km)까지 계산해
#   "짧은 로그에서 과대평가" 리스크를 방지하는 흐름을 보여준다.

# ==============================
# 1. 데이터 로드
# ==============================
# 2025-08-25
# 상대경로는 실행 위치(CWD)에 따라 깨질 수 있다.
# notebook/ 폴더에서 실행하는 케이스를 고려해, repo 기준 경로를 안전하게 잡는다.
csv_path = Path.cwd().parent / "data" / "driving_log.csv"

# 2025-08-25
# 파일이 없으면 조용히 넘어가지 말고 즉시 실패시키는 게 재현/검증에 안전하다.
if not csv_path.exists():
    raise FileNotFoundError(f"driving_log.csv not found: {csv_path}")

df = pd.read_csv(csv_path)

# 2025-08-26
# 스키마가 다르면 뒤 로직이 전부 무의미해지므로, 필수 컬럼을 초반에 강제 체크한다.
required_cols = {"time_sec", "speed_kmh", "accel_mps2"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

# ==============================
# 2. 임계값 설정 (m/s^2)
# ==============================
# 2025-08-26
# 임계값은 프로젝트 간 비교를 위해 고정값으로 유지.
HARD_ACCEL_THRESHOLD = 3.0
HARD_BRAKE_THRESHOLD = -3.0

# ==============================
# 3. 급가속 / 급제동 이벤트 탐지
# ==============================
# 2025-08-27
# 현재 버전은 "샘플 단위 카운트"로 단순화.
# (연속 구간을 하나의 이벤트로 묶는 segmenting은 고도화 과제로 남긴다.)
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]

hard_accel_count = len(hard_accel_df)
hard_brake_count = len(hard_brake_df)
total_hard_events = hard_accel_count + hard_brake_count

# ==============================
# 4. 주행 시간 / 거리 계산
# ==============================
# 2025-08-28
# 거리 계산은 평균속도 기반 근사.
# (파이프라인 흐름 확인 목적이며, 필요 시 time_sec 적분 방식으로 개선 가능)
total_time_sec = df["time_sec"].max() - df["time_sec"].min()

avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# ==============================
# 5. 이벤트 빈도 계산 (안정성 처리)
# ==============================
# 2025-08-29
# 짧은 로그(예: 몇 초)는 분모가 너무 작아 빈도 지표가 튀거나,
# 반대로 현재처럼 60초 미만이면 빈도 계산 자체가 의미가 없다.
# 그래서 최소 시간/거리 조건을 통과 못하면 None으로 처리한다.
events_per_min = (total_hard_events / (total_time_sec / 60)) if total_time_sec >= 60 else None
events_per_km = (total_hard_events / distance_km) if distance_km > 0.5 else None

# ==============================
# 6. 분류 함수
# ==============================
# 2025-08-30
# Count 기준 분류는 단순하지만, "아주 짧은 로그"에서 과대평가될 수 있다.
def classify_by_count(total_events):
    if total_events <= 2:
        return "Smooth Driver"
    elif total_events <= 5:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

# 2025-08-30
# Rate 기준은 주행 길이를 보정하지만, 데이터 부족(None) 상황은 별도로 처리한다.
def classify_by_rate(events_per_min):
    if events_per_min is None:
        return "Insufficient Data"
    elif events_per_min < 3:
        return "Smooth Driver"
    elif events_per_min < 6:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

# ==============================
# 7. 분류 결과
# ==============================
driver_type_count = classify_by_count(total_hard_events)
driver_type_rate = classify_by_rate(events_per_min)

# ==============================
# 8. 최종 출력
# ==============================
# 2025-08-31
# 최종 결과는 점수/등급만 보여주지 말고,
# "왜 그렇게 나왔는지" 중간 지표를 같이 출력한다.
print("Driver Behavior Analysis Result")
print("=" * 45)
print(f"Data Path               : {csv_path}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.2f}")
print("-" * 45)
print(f"Hard Acceleration Events : {hard_accel_count}")
print(f"Hard Braking Events      : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print("-" * 45)
print(f"Events per Minute        : {events_per_min:.2f}" if events_per_min is not None else
      "Events per Minute        : Insufficient Data")
print(f"Events per km            : {events_per_km:.2f}" if events_per_km is not None else
      "Events per km            : Insufficient Data")
print("-" * 45)
print(f"Classification (Count)   : {driver_type_count}")
print(f"Classification (Rate)    : {driver_type_rate}")


Driver Behavior Analysis Result
-----------------------------------
Hard Acceleration Events : 3
Hard Braking Events      : 3
Total Hard Events        : 6
Total Time (sec)         : 9.0
Events per Minute        : Insufficient Data
Events per km            : Insufficient Data
-----------------------------------
Classification (Count)   : Aggressive Driver
Classification (Rate)    : Insufficient Data
